In [54]:
# Importar bibliotecas necessárias
import pandas as pd
from manim import *
from IPython.display import display, Markdown

# Ler o arquivo Markdown no formato de tabela
file_path = "../scenes/arvore_conceitual.md"
data = pd.read_csv(file_path, sep="|", skiprows=1, usecols=[1, 2, 3, 4])

# Renomear colunas e limpar espaços extras
data.columns = ["id", "text", "shape", "parent"]
data = data.apply(lambda x: x.str.strip() if x.dtype == "object" else x)

# Exibir título e tabela
display(Markdown("### Tabela da Árvore Conceitual"))
display(data)

# Função para criar os nós e as conexões
class ArvoreConceitual(Scene):
    def construct(self):
        # Dicionário para armazenar os objetos dos nós
        nodes = {}
        
        # Criar os nós com base na tabela
        for _, row in data.iterrows():
            if row["shape"] == "circle":
                node = Circle().scale(0.5).set_fill(BLUE, opacity=0.5)
            elif row["shape"] == "rectangle":
                node = Square().scale(0.5).set_fill(RED, opacity=0.5)
            elif row["shape"] == "ellipse":
                node = Ellipse(width=2.0, height=1.0).set_fill(GREEN, opacity=0.5)
            else:
                node = Dot()
            
        node_text = Text(row["text"]).scale(0.5)
        group = VGroup(node, node_text).arrange(UP)

        parent_index = nodes.get(int(row["parent"]))
        console.log(parent_index)

        if parent_index == "null" or parent_index is None:
            group.move_to(ORIGIN)
        else:
            parent_node = nodes.get(int(row["parent"]))
            group.next_to(parent_node, DOWN, buff=1)
        
        self.add(group)
        self.play(Create(group))
        nodes[row["id"]] = group

        # Conectar os nós com linhas
        for _, row in data.iterrows():
            if row["parent"] != "null":
                parent_id = int(row["parent"])
                self.add(Line(nodes[parent_id].get_center(), nodes[row["id"]].get_center()))

def build_tree(df):
    """
    Converte o DataFrame em uma estrutura hierárquica.
    """
    nodes = {}
    for _, row in df.iterrows():
        node = {
            "text": row["text"],
            "shape": row["shape"],
            "children": []
        }
        nodes[row["id"]] = node

    # Construir hierarquia
    for _, row in df.iterrows():
        parent_id = row["parent"]
        if parent_id != "null":
            parent_id = int(parent_id)
            nodes[parent_id]["children"].append(nodes[row["id"]])
    
    # Retorna o nó raiz
    return nodes[1]

raiz = build_tree(data)
tree_data = {
    "type": "ArvoreConceitual",
    "title": "Árvore Conceitual",
    "nodes": raiz
}

tree_data


### Tabela da Árvore Conceitual

,id,text,shape,parent
0,1,Raiz,circle,null
1,2,Galho 1,rectangle,1
2,3,Galho 2,rectangle,1
3,4,Folha 2.1,ellipse,2
4,5,Folha 2.2,circle,2
5,6,Galho 3,circle,1
6,7,Folha 3.1,Square,6


{'type': 'ArvoreConceitual',
 'title': 'Árvore Conceitual',
 'nodes': {'text': 'Raiz',
  'shape': 'circle',
  'children': [{'text': 'Galho 1',
    'shape': 'rectangle',
    'children': [{'text': 'Folha 2.1', 'shape': 'ellipse', 'children': []},
     {'text': 'Folha 2.2', 'shape': 'circle', 'children': []}]},
   {'text': 'Galho 2', 'shape': 'rectangle', 'children': []},
   {'text': 'Galho 3',
    'shape': 'circle',
    'children': [{'text': 'Folha 3.1', 'shape': 'Square', 'children': []}]}]}}

In [55]:
# Renderizar a cena
config.media_width = "75%"
console.log(tree_data)
scene = ArvoreConceitual()
scene.render()

# Mostrar o vídeo renderizado
from IPython.display import Video
Video("media/videos/arvore_conceitual/480p15/ArvoreConceitual.mp4")


[19:17:50] {                                                                                        ]8;id=432645;file:///tmp/ipykernel_12028/1093343438.py\1093343438.py]8;;\:]8;id=341670;file:///tmp/ipykernel_12028/1093343438.py#3\3]8;;\
               'type': 'ArvoreConceitual',                                                                         
               'title': 'Árvore Conceitual',                                                                       
               'nodes': {                                                                                          
                   'text': 'Raiz',                                                                                 
                   'shape': 'circle',                                                                              
                   'children': [                                                                                   
                       {                                                                                           
                           'text': 'Galho 1',                                                                      
                           'shape': 'rectangle',                                                                   
                           'children': [                                                                           
                               {'text': 'Folha 2.1', 'shape': 'ellipse', 'children': []},                          
                               {'text': 'Folha 2.2', 'shape': 'circle', 'children': []}                            
                           ]                                                                                       
                       },                                                                                          
                       {'text': 'Galho 2', 'shape': 'rectangle', 'children': []},                                  
                       {                                                                                           
                           'text': 'Galho 3',                                                                      
                           'shape': 'circle',                                                                      
                           'children': [{'text': 'Folha 3.1', 'shape': 'Square', 'children': []}]                  
                       }                                                                                           
                   ]                                                                                               
               }                                                                                                   
           }                                                                                                       

           None                                                                                    ]8;id=636356;file:///tmp/ipykernel_12028/3006157370.py\3006157370.py]8;;\:]8;id=208202;file:///tmp/ipykernel_12028/3006157370.py#39\39]8;;\

[01/17/25 19:19:13] INFO     Animation 0 : Partial movie file written in                   ]8;id=773921;file:///home/ec2-user/.local/lib/python3.9/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=174177;file:///home/ec2-user/.local/lib/python3.9/site-packages/manim/scene/scene_file_writer.py#527\527]8;;\
                             '/home/ec2-user/moviemaker/templates/media/videos/1080p60/par                         
                             tial_movie_files/ArvoreConceitual/3977891868_1870908798_14128                         
                             41409.mp4'                                                                            

KeyError: 1